<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202501/blob/main/whisky/02_script_EDA_whisky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import plotly.express as px
import re

In [3]:
path = '/content/drive/MyDrive/Teaching/UPSA/2025.01/MOD/whisky/data-whisky.parquet'

In [4]:
df = pd.read_parquet(path)

# Situación competitiva

## Lealtad

In [5]:
df_lealtad = df['¿Cuál es la marca que más compra?'] \
.value_counts() \
.reset_index() \
.rename(columns = {
    '¿Cuál es la marca que más compra?': 'Marca',
    'count': 'n'
})

In [6]:
df_lealtad['Porcentaje'] = df_lealtad['n'] / df_lealtad['n'].sum()
df_lealtad['Porcentaje_txt'] = df_lealtad['Porcentaje'].apply(lambda x: str(round(x*100, 0)) + '%')

In [ ]:
df_lealtad

,Marca,n,Porcentaje,Porcentaje_txt
0,Johnny Walker,43,0.524390,52.0%
1,Jack Daniel's,18,0.219512,22.0%
2,Chivas Regal,14,0.170732,17.0%
3,Old Parr,6,0.073171,7.0%
4,Ninguno,1,0.012195,1.0%


In [ ]:
px.bar(
    df_lealtad,
    x = 'Marca',
    y = 'Porcentaje',
    text = 'Porcentaje_txt',
    title = '¿Cuál es la marca que más compra?',
    # color = 'Marca'
).update_layout(
    template = 'plotly_white',
    yaxis_showticklabels = False,
    yaxis_showgrid = False,
    xaxis_title = ''
)

## Conocimiento

In [ ]:
[col for col in df.columns if col.startswith('Conocimiento')]

['Conocimiento1', 'Conocimiento2', 'Conocimiento3', 'Conocimiento4']

In [ ]:
df_conocimiento_long = df.melt(
    id_vars = [],
    value_vars = [col for col in df.columns if col.startswith('Conocimiento')],
    var_name = 'KPI',
    value_name= 'Marca'
).dropna()

In [ ]:
df_conocimiento_long.head()

,KPI,Marca
0,Conocimiento1,Johnny Walker
1,Conocimiento1,Johnny Walker
2,Conocimiento1,Chivas Regal
3,Conocimiento1,Chivas Regal
4,Conocimiento1,Chivas Regal


In [ ]:
df_conocimiento = df_conocimiento_long['Marca'] \
.value_counts() \
.reset_index() \
.rename(columns = {
    'count': 'n'
})

In [ ]:
df_conocimiento

,Marca,n
0,Johnny Walker,71
1,Chivas Regal,61
2,Jack Daniel's,56
3,Old Parr,44


In [ ]:
df_conocimiento['Porcentaje'] = df_conocimiento['n'] / len(df)
df_conocimiento['Porcentaje_txt'] = df_conocimiento['Porcentaje'].apply(lambda x: str(round(x*100, 0)) + '%')

In [ ]:
df_conocimiento

,Marca,n,Porcentaje,Porcentaje_txt
0,Johnny Walker,71,0.865854,87.0%
1,Chivas Regal,61,0.743902,74.0%
2,Jack Daniel's,56,0.682927,68.0%
3,Old Parr,44,0.536585,54.0%


In [ ]:
px.bar(
    df_conocimiento,
    x = 'Marca',
    y = 'Porcentaje',
    text = 'Porcentaje_txt',
    title = '¿Cuál de estas marcas conoce?',
    # color = 'Marca'
).update_layout(
    template = 'plotly_white',
    yaxis_showticklabels = False,
    yaxis_showgrid = False,
    xaxis_title = ''
)

## Consideración

In [7]:
df['Consideración Chivas'].unique()

array(['NUNCA la consideraría', 'Creo que NO la consideraría',
       'Creo que SÍ la consideraría', 'Sería mi 1ra opción'], dtype=object)

In [12]:
consideracion_t2b = ['Creo que SÍ la consideraría', 'Sería mi 1ra opción']
consideracion_t2b

['Creo que SÍ la consideraría', 'Sería mi 1ra opción']

In [11]:
consideracion_cols = [col for col in df.columns if col.startswith('Consideración')]
consideracion_cols

['Consideración Chivas',
 'Consideración Johnny Walker',
 'Consideración Old Parr',
 "Consideración Jack Daniel's"]

In [15]:
# Demo individual
df['Consideración Johnny Walker'].apply(lambda x: x in consideracion_t2b).sum()

np.int64(57)

In [21]:
# Demo individual
re.search(r'(Consideración\s)(.+)', 'Consideración Johnny Walker').group(2)

'Johnny Walker'

In [23]:
consideracion_dict = []

for col in consideracion_cols:
  Total = len(df)
  Considera = df[col].apply(lambda x: x in consideracion_t2b).sum()
  Porcentaje = Considera / Total
  Porcentaje_txt = str(round(Porcentaje*100, 0)) + '%'
  Marca = re.search(r'(Consideración\s)(.+)', col).group(2)
  consideracion_dict.append({
      'Marca': Marca,
      'n': Considera,
      'Porcentaje': Porcentaje,
      'Porcentaje_txt': Porcentaje_txt
  })

In [25]:
df_consideracion = pd.DataFrame(consideracion_dict)
df_consideracion

,Marca,n,Porcentaje,Porcentaje_txt
0,Chivas,30,0.365854,37.0%
1,Johnny Walker,57,0.695122,70.0%
2,Old Parr,34,0.414634,41.0%
3,Jack Daniel's,58,0.707317,71.0%
